# Silver Notebook

## The following Notebook will aim to 
- Perform data transformations on the data 
- Make the data ready for the Gold layer

### Customers Data 
Transformations to do on the volume from Bronze Schema
- Remove the records with Null Customer_id 
- Remove exact Duplicate Records
- Remove the Duplicate Records based on created_timestamp 
- CAST the columns to the correct Data Type 
- Write the transformed data to the silver schema

In [0]:
%sql
SELECT *
FROM gizmoboxcatalog.bronzeschema.v_customers
WHERE customer_id IS NOT NULL;

In [0]:
%sql
SELECT DISTINCT 
  *
FROM gizmoboxcatalog.bronzeschema.v_customers
WHERE CUSTOMER_ID IS NOT NULL
ORDER BY CUSTOMER_ID ASC;

In [0]:
%sql
CREATE OR REPLACE VIEW gizmoboxcatalog.silverschema.v_customers_distinct 
AS
SELECT DISTINCT 
  *
FROM gizmoboxcatalog.bronzeschema.v_customers
WHERE CUSTOMER_ID IS NOT NULL
ORDER BY CUSTOMER_ID ASC;

In [0]:
%sql
SELECT 
  CUSTOMER_ID, 
  MAX(created_timestamp) AS max_created_timestamp
FROM gizmoboxcatalog.silverschema.v_customers_distinct
GROUP BY customer_id;

In [0]:
%sql
WITH CTE AS (
  SELECT 
  CUSTOMER_ID, 
  MAX(created_timestamp) AS max_created_timestamp
  FROM gizmoboxcatalog.silverschema.v_customers_distinct
  GROUP BY customer_id
)
SELECT *
FROM gizmoboxcatalog.silverschema.v_customers_distinct AS T1
JOIN CTE AS T2
ON 
  T1.CUSTOMER_ID = T2.CUSTOMER_ID
  AND 
  T1.created_timestamp = T2.max_created_timestamp;

In [0]:
%sql
WITH CTE AS (
  SELECT 
  CUSTOMER_ID, 
  MAX(created_timestamp) AS max_created_timestamp
  FROM gizmoboxcatalog.silverschema.v_customers_distinct
  GROUP BY customer_id
)
SELECT 
      CAST(T1.created_timestamp AS TIMESTAMP) AS created_timestamp, 
      T1.customer_id, 
      T1.customer_name, 
      CAST(T1.date_of_birth AS DATE) AS date_of_birth, 
      T1.email, 
      CAST(T1.member_since AS DATE) AS member_since, 
      T1.telephone
FROM gizmoboxcatalog.silverschema.v_customers_distinct AS T1
JOIN CTE AS T2
ON 
  T1.CUSTOMER_ID = T2.CUSTOMER_ID
  AND 
  T1.created_timestamp = T2.max_created_timestamp;

In [0]:
%sql
CREATE OR REPLACE TABLE gizmoboxcatalog.silverschema.customers
AS
WITH CTE AS (
  SELECT 
  CUSTOMER_ID, 
  MAX(created_timestamp) AS max_created_timestamp
  FROM gizmoboxcatalog.silverschema.v_customers_distinct
  GROUP BY customer_id
)
SELECT 
      CAST(T1.created_timestamp AS TIMESTAMP) AS created_timestamp, 
      T1.customer_id, 
      T1.customer_name, 
      CAST(T1.date_of_birth AS DATE) AS date_of_birth, 
      T1.email, 
      CAST(T1.member_since AS DATE) AS member_since, 
      T1.telephone
FROM gizmoboxcatalog.silverschema.v_customers_distinct AS T1
JOIN CTE AS T2
ON 
  T1.CUSTOMER_ID = T2.CUSTOMER_ID
  AND 
  T1.created_timestamp = T2.max_created_timestamp;

  SELECT *
  FROM gizmoboxcatalog.silverschema.customers;

### Transforming the Payments Data 
- Extract the Date and Time from Payments_timestamp and create a new column payment_date and payment_time 
- Map payment_status to contain descriptive values (1 - Success, 2 - Pending, 3 - Cancelled, 4 - Failed) 
- Write the transformed data to the silver schema 

In [0]:
%sql
SELECT
  payment_id, 
  order_id, 
  CAST(DATE_FORMAT(PAYMENT_TIMESTAMP, 'yyyy-MM-dd') AS DATE) AS payment_date, 
  DATE_FORMAT(PAYMENT_TIMESTAMP, 'HH:mm:ss') AS payment_time, 
  payment_status, 
  payment_method
FROM gizmoboxcatalog.bronzeschema.payments;

In [0]:
%sql
SELECT
  payment_id, 
  order_id, 
  CAST(DATE_FORMAT(PAYMENT_TIMESTAMP, 'yyyy-MM-dd') AS DATE) AS payment_date, 
  DATE_FORMAT(PAYMENT_TIMESTAMP, 'HH:mm:ss') AS payment_time, 
  CASE payment_status
    WHEN 1 THEN 'Success'
    WHEN 2 THEN 'Pending'
    WHEN 3 THEN 'Cancelled'
    WHEN 4 THEN 'Failed'
    ELSE 'Unkown'
  END AS payment_status, 
  payment_method
FROM gizmoboxcatalog.bronzeschema.payments;

In [0]:
%sql
CREATE OR REPLACE TABLE gizmoboxcatalog.silverschema.payments
AS
SELECT
  payment_id, 
  order_id, 
  CAST(DATE_FORMAT(PAYMENT_TIMESTAMP, 'yyyy-MM-dd') AS DATE) AS payment_date, 
  DATE_FORMAT(PAYMENT_TIMESTAMP, 'HH:mm:ss') AS payment_time, 
  CASE payment_status
    WHEN 1 THEN 'Success'
    WHEN 2 THEN 'Pending'
    WHEN 3 THEN 'Cancelled'
    WHEN 4 THEN 'Failed'
    ELSE 'Unkown'
  END AS payment_status, 
  payment_method
FROM gizmoboxcatalog.bronzeschema.payments;

SELECT *
FROM gizmoboxcatalog.silverschema.payments;

### Transforming the Addresses Data 
- Create one record for each customer with 2 sets of address columns, 1 for shipping and 1 for billing 
- Write the transformed data to the silver schema

In [0]:
%sql
SELECT
  *
FROM gizmoboxcatalog.bronzeschema.v_addresses;

In [0]:
%sql
SELECT *
FROM 
(
  SELECT 
    CUSTOMER_ID, 
    ADDRESS_TYPE, 
    ADDRESS_LINE_1, 
    CITY, 
    STATE, 
    POSTCODE
  FROM gizmoboxcatalog.bronzeschema.v_addresses
)
PIVOT (
  MAX(address_line_1) AS address_line_1, 
  MAX(city) AS city, 
  MAX(state) AS state, 
  MAX(postcode) AS postcode
  FOR address_type IN ('shipping', 'billing')
)

In [0]:
%sql
CREATE OR REPLACE TABLE gizmoboxcatalog.silverschema.addresses
AS 
SELECT *
FROM 
(
  SELECT 
    CUSTOMER_ID, 
    ADDRESS_TYPE, 
    ADDRESS_LINE_1, 
    CITY, 
    STATE, 
    POSTCODE
  FROM gizmoboxcatalog.bronzeschema.v_addresses
)
PIVOT (
  MAX(address_line_1) AS address_line_1, 
  MAX(city) AS city, 
  MAX(state) AS state, 
  MAX(postcode) AS postcode
  FOR address_type IN ('shipping', 'billing')
);

SELECT *
FROM gizmoboxcatalog.silverschema.addresses;

### Transforming the Orders Data 
- Extract Top Level Column Values 
- Extract Array Elements 
- Extract Names Column Values 
- Cast Columns Values to Specific Data Type

In [0]:
%sql
SELECT *
FROM gizmoboxcatalog.bronzeschema.v_orders;

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW tv_orders_fixed 
AS 
SELECT 
  value, 
  regexp_replace(value, '"order_date": (\\d{4}-\\d{2}-\\d{2})', '"order_date": "\$1"') AS fixed_value
FROM gizmoboxcatalog.bronzeschema.v_orders

In [0]:
%sql
SELECT 
    schema_of_json(fixed_value)
FROM tv_orders_fixed
LIMIT 1; 

In [0]:
%sql
SELECT
    FROM_JSON(
      fixed_value,
      'STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>'
      ) AS fixed_value
FROM tv_orders_fixed;

In [0]:
%sql
CREATE OR REPLACE TABLE gizmoboxcatalog.silverschema.orders AS
SELECT
  fixed_value.customer_id,
  fixed_value.order_id,
  fixed_value.order_date,
  fixed_value.order_status,
  fixed_value.payment_method,
  fixed_value.total_amount,
  fixed_value.transaction_timestamp,
  item.category,
  item.details.brand,
  item.details.color,
  item.item_id,
  item.name           AS item_name,
  item.price,
  item.quantity
FROM gizmoboxcatalog.silverschema.orders_json
LATERAL VIEW explode(fixed_value.items) AS item;


In [0]:
%sql
SELECT *
FROM gizmoboxcatalog.silverschema.orders;

In [0]:
%sql
-- Flatten nested JSON into normal table
CREATE OR REPLACE TABLE gizmoboxcatalog.silverschema.orders AS
SELECT
  customer_id,
  order_id,
  order_date,
  order_status,
  payment_method,
  total_amount,
  transaction_timestamp,
  fixed_value.category,
  fixed_value.details.brand       AS brand,
  fixed_value.details.color       AS color,
  fixed_value.item_id,
  fixed_value.name                AS item_name,
  fixed_value.price,
  fixed_value.quantity
FROM gizmoboxcatalog.silverschema.orders_json
LATERAL VIEW explode(fixed_value.items) AS item;
